# with updated lodes data

In [11]:
import pandas as pd
import geopandas as gpd
import json
import requests
import numpy as np
from shapely.geometry import Point

In [12]:
# def traveltimeTransit(latHome, lonHome,latWork,lonWork, time)->int:
#     request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
#     + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
#     +time+"&maxWalkDistance=750" # chaneg distance 
#     r = requests.get(request)
#     data=r.json()
#     if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
#         return (data['plan']['itineraries'])
#     else:
#         return np.nan

In [13]:
# http://localhost:8080/otp/routers/default/plan?fromPlace=str(latHome),str(lonHome)&toPlace=str(latWork),&+&mode=WALK,TRANSIT&date=05-04-2022&time=10:00am&maxWalkDistance=750

In [23]:
def traveltimeTransit2(df, i, latHome, lonHome,latWork,lonWork, time)->int:
    walk_distance = 10000
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
    +time+"&maxWalkDistance="+str(walk_distance)
    # print(request)
    r = requests.get(request)
    data=r.json()
 
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        df.loc[i, 'transit_time'] = int(data['plan']['itineraries'][0]['duration'])
        df.loc[i, 'transfers'] = int(data['plan']['itineraries'][0]['transfers'])
        legs = pd.DataFrame(data['plan']['itineraries'][0]['legs'])
        # df.loc[i, 'trips'] = np.array(legs['tripId'].dropna().to_list())
        # df.loc[i, 'trips'] = df.apply(lambda r: tuple(legs['tripId'].dropna()), axis=1).apply(np.array)
        try:
            df.loc[i, 'trip1'] = legs['tripId'].dropna().iloc[0].split(':')[1]
        except:pass
        try:
            df.loc[i, 'trip2'] = legs['tripId'].dropna().iloc[0].split(':')[1]
        except:pass
        try:
            df.loc[i, 'trip3'] = legs['tripId'].dropna().iloc[0].split(':')[2]
        except:pass
        try:
            df.loc[i, 'route1'] = legs.route[legs.route != ''].iloc[0]
        except:pass
        try:
            df.loc[i, 'route2'] = legs.route[legs.route != ''].iloc[1]
        except:pass
        try:
            df.loc[i, 'route3'] = legs.route[legs.route != ''].iloc[2]
        except:pass
        df.loc[i, 'bus_capacity_upto_2hours'] = len(data['plan']['itineraries'])*30

        
    else:
        df.loc[i, 'transit_time'] =  np.nan
        df.loc[i, 'transfers'] = np.nan
        # df.loc[i, 'trips'] = np.nan
        df.loc[i, 'trip1'] =  np.nan
        df.loc[i, 'trip2'] =  np.nan
        df.loc[i, 'trip3'] =  np.nan
        df.loc[i, 'route1'] =  np.nan
        df.loc[i, 'route2'] =  np.nan
        df.loc[i, 'route3'] =  np.nan
        df.loc[i, 'bus_capacity_upto_2hours'] = np.nan

    
    

In [24]:
def traveltimeDrive(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,CAR&date=05-04-2021&time="\
    +time+"&maxWalkDistance=750"
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [25]:
def traveltimeWalk(latHome, lonHome,latWork,lonWork, time)->int:
    walk_distance = 50000
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK&date=05-04-2021&time="\
    +time+"&maxWalkDistance="+str(walk_distance)
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [26]:
def get_stop_url(geom):
    # print(df)
    # geom = df.geometry
    initial='http://127.0.0.1:5000/nearest/v1/driving/'
    end='?number=1'
    point = str(geom.x)+","+str(geom.y)
    requests2 = []
    requests2.append(initial+point+end)
    return requests2
    

In [27]:
import time
def nearby_points(reqs):
    url=reqs[0]
    #to remove any connection timeouts and port usage exceptions
    # r=None
    # l=None
    try:
        r=requests.get(url)
        data=r.json()
        l=Point(data['waypoints'][0]['location'])
        return l
    except:
        time.sleep(60)
        
    # return l

## input: lodes od data

In [28]:
# od_lodes = pd.read_parquet('data/lodes_od_based_on_job.parquet')
od_lodes = pd.read_parquet('./lodes_combs_chatta.parquet')

In [29]:
od_lodes.head()

,h_geocode,w_geocode,total_jobs,home_loc_lat,home_loc_lon,work_loc_lat,work_loc_lon,go_time,go_time_str,return_time,return_time_str
0,470650113214,470650004001,133.0,35.035038,-85.143375,35.044494,-85.270026,1900-01-01 07:25:30,07:25:30,1900-01-01 17:48:45,17:48:45
1,470650113214,470650004001,133.0,35.031954,-85.139541,35.039947,-85.268589,1900-01-01 08:27:00,08:27:00,1900-01-01 17:07:30,17:07:30
2,470650113214,470650004001,133.0,35.024332,-85.141216,35.039947,-85.268589,1900-01-01 07:26:30,07:26:30,1900-01-01 16:17:45,16:17:45
3,470650113214,470650004001,133.0,35.030575,-85.154980,35.040336,-85.269585,1900-01-01 07:47:00,07:47:00,1900-01-01 16:05:00,16:05:00
4,470650113214,470650004001,133.0,35.022464,-85.143388,35.040336,-85.269585,1900-01-01 07:25:30,07:25:30,1900-01-01 16:30:15,16:30:15


In [30]:
from tqdm.notebook import tqdm_notebook

for i, val in tqdm_notebook(od_lodes.iterrows(), total=od_lodes.shape[0]):
    traveltimeTransit2(od_lodes, i, od_lodes.iloc[i].home_loc_lat, od_lodes.iloc[i].home_loc_lon, od_lodes.iloc[i].work_loc_lat, od_lodes.iloc[i].work_loc_lon, od_lodes.iloc[i].go_time_str+'am')
    od_lodes.loc[i, 'drive_time_fn'] = traveltimeDrive(od_lodes.iloc[i].home_loc_lat, od_lodes.iloc[i].home_loc_lon, od_lodes.iloc[i].work_loc_lat, od_lodes.iloc[i].work_loc_lon, od_lodes.iloc[i].go_time_str+'am')
    od_lodes.loc[i, 'walk_time_fn'] = traveltimeWalk(od_lodes.iloc[i].home_loc_lat, od_lodes.iloc[i].home_loc_lon, od_lodes.iloc[i].work_loc_lat, od_lodes.iloc[i].work_loc_lon, od_lodes.iloc[i].go_time_str+'am' )
    od_lodes.loc[i, 'transit_score'] = np.divide(od_lodes.loc[i, 'drive_time_fn'], od_lodes.loc[i, 'transit_time'])

  0%|          | 0/31527 [00:00<?, ?it/s]

In [31]:
od_lodes.to_csv('lodes_od_chatta_10k.csv', index=False)
# [hw_tracts_geom['TRACTCE20_home'] == hw_tracts_geom['TRACTCE20_work']].transit_score.dropna()

In [33]:
ham_within_chatta = pd.read_csv('ham_within_chatta.csv')
ham_within_chatta.GEOID = ham_within_chatta.GEOID.astype(str)
ham_within_chatta['name'] = ham_within_chatta.GEOID.apply(lambda x: 'Block Group '+str(x[-1]) +', Census Tract '+str(x[6:9]).lstrip('0')+'.'+str(x[9:-1])+', Hamilton County, Tennessee' if str(x[9:-1]) != '00' else 'Block Group '+str(x[-1]) +', Census Tract '+str(x[6:9]).lstrip('0')+', Hamilton County, Tennessee')

In [36]:
od_lodes.merge(ham_within_chatta[['GEOID', 'geometry']], left_on='h_geocode', right_on='GEOID').to_csv('lodes_od_chatta_10k_ts_2021_10_gtfs.csv', index=False)
od_lodes.groupby('h_geocode').mean().reset_index().merge(ham_within_chatta[['GEOID', 'geometry']], left_on='h_geocode', right_on='GEOID').to_csv('chatta_ts_mean_2021_10k.csv', index=False)